<a href="https://colab.research.google.com/github/VIS-WA/Duplicate-Question-Detection-in-Stack-Overflow/blob/main/PrimaryPhysics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/study"

/content/drive/MyDrive/study


## Libraries

In [3]:
# libraries
import nltk #NLP lib
import numpy as np # numpy for arrays
import pandas as pd # pandas lib for data handling
from nltk.corpus import stopwords # common eng sstopwords
from nltk.stem import PorterStemmer #porter stemmer algo
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup # HTML to txt
import os
import sys
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
ps = PorterStemmer()
sw_nltk = stopwords.words('english') # bag of all common english stop words
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Supporting Functions

In [4]:
import re
def TokStem(text,j): # Tokenise and Stem the given text
    stoplist = set(nltk.corpus.stopwords.words('english'))
    # text = [contraction[word.islower()] for word in text.split(" ") if word.islower() in contractions.keys() ]
    # print(text)
    
    text =  nltk.word_tokenize(text.lower())
    # text = re.split(',text)    
    # print(text)
    
    words = text #[word for word in text if word.isalpha()]
    ltz = WordNetLemmatizer()
    # words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final1 = []
    final = []
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~``'''
    contract_strings = ["'s", "\"","'ve"]
    for word in words:
      if word not in stoplist and word not in punc and word not in contract_strings and word!='"':
        
        final1.append(ltz.lemmatize(word))
        final.append(ps.stem(word))
    # print(final2)
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I C++ . first-day \"met! 'her' will her's not she_was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)
print(x)

['c++', 'first-day', 'met', "'her", 'she_wa', 'quiet', 'remain', 'quiet', 'entir', 'two', 'hour', 'long', 'journey', 'stoni', 'brook', 'new', 'york']


In [ ]:
def TokStem(text,j): # Tokenise and Stem the given text
    words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final = []
    for w in words:
        final.append(ps.stem(w))
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I first met her she was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)

In [5]:
def hbt(text): # convert HTML body to text
    x = BeautifulSoup(text).get_text()
    # print(x)
    return x

In [6]:
def rmTags(text): # convert tags to text
    st,out = [],[]
    for i in text:
        if i == '<':
            st = []
        elif i == '>':
            st += ' '
            out += st
        else:
            st += i
    return "".join(out[:-1]) # remove the last space

### cleaning duplicates

In [8]:
df = pd.read_csv("PhysicsDataset.csv")

te = []
for i in range(len(df)): # pickout original qns
    x = int(df[['OId']].loc[i])
    if x in te:
        df.drop([i], inplace = True) # remove rows
    else:
        te.append(x)
# print(Q2.shape)
df = df.reset_index(drop=True) # restore the indices
df.to_csv('PhysicsDatasetCleaned.csv') # export to CSV

## Loading the data 

In [12]:
df = pd.read_csv('PhysicsDatasetCleaned.csv')
df

,Unnamed: 0,DId,DTitle,DTags,DBody,OId,OTitle,OTags,OBody
0,0,131129,How to explain spin of electron?,<quantum-mechanics><particle-physics><angular-...,"<p>How can we explain spin of electron, or the...",1,What is spin as it relates to subatomic partic...,<quantum-mechanics><particle-physics><angular-...,<p>I often hear about subatomic particles havi...
1,1,82125,Is the universe immediately dependent upon the...,<time><determinism>,<p>Given complete knowledge of the precise sta...,7,Will Determinism be ever possible?,<quantum-mechanics><quantum-interpretations><h...,<p>What are the main problems that we need to ...
2,2,217508,Proof of Hamilton's principle,<lagrangian-formalism><action><variational-cal...,<p>Is there a anything like a proof of Hamilto...,9,Why should an action integral be stationary? O...,<lagrangian-formalism><variational-principle><...,<p>Hamilton's principle states that a dynamic ...
3,3,486529,Will string theory ever be provable?,<string-theory>,<p>Some time ago I saw a documentary in which ...,15,What experiment would disprove string theory?,<string-theory><experimental-physics>,<p>I know that there's big controversy between...
4,4,175205,The interaction between waves and particles ac...,<waves><electromagnetic-radiation><scattering>,<p>Why do EM waves with a large wavelength lik...,17,Why does the sky change color? Why is the sky ...,<optics><visible-light><scattering><atmospheri...,<p>Why does the sky change color? Why is the s...
...,...,...,...,...,...,...,...,...,...
6105,6105,679483,How fast does an electromagnetic field come in...,<electromagnetism><magnetic-fields><electric-c...,"<p>In <a href=""https://www.youtube.com/watch?v...",678359,How is the answer to this question $1/c$ seconds?,<electromagnetism><speed-of-light><conductors>...,"<p>This is <a href=""https://www.youtube.com/wa..."
6106,6106,679870,How do mathematicians derive equations and for...,<mathematics>,<p>How do scientists use arithmetic to derive ...,679619,How do scientists derive equations and formula...,<mathematics>,<p>How do scientists use arithmetic to derive ...
6107,6107,680469,Magnetic lines of force,<electrostatics><electric-fields><vector-fields>,<p>There are lines of magnetic force released ...,680321,Forces and magnetic lines of force,<forces><newtonian-gravity><magnetic-fields><e...,<p>Magnet releases the magnetic lines of force...
6108,6108,679556,Escape velocity at 45 degrees near a black hole,<black-holes><photons><escape-velocity>,<p>what if the formula for the escape velocity...,680400,Escape velocity at an angle near a black hole,<black-holes><escape-velocity>,<p>What's the formula for the velocity require...


## Data Preprocessing

In [13]:
N1 = len(df)
N2 = 300
# duplicate questions for training (with sorting)
Q2 = df[['OId','OTitle','OBody','OTags']].loc[0:N1-1].copy() # original qns
Q1 = df[['DId','DTitle','DBody','DTags']].loc[0:N2-1].copy() # doop qns
print("Duplicate  question set")
Q1


Duplicate  question set


,DId,DTitle,DBody,DTags
0,131129,How to explain spin of electron?,"<p>How can we explain spin of electron, or the...",<quantum-mechanics><particle-physics><angular-...
1,82125,Is the universe immediately dependent upon the...,<p>Given complete knowledge of the precise sta...,<time><determinism>
2,217508,Proof of Hamilton's principle,<p>Is there a anything like a proof of Hamilto...,<lagrangian-formalism><action><variational-cal...
3,486529,Will string theory ever be provable?,<p>Some time ago I saw a documentary in which ...,<string-theory>
4,175205,The interaction between waves and particles ac...,<p>Why do EM waves with a large wavelength lik...,<waves><electromagnetic-radiation><scattering>
...,...,...,...,...
295,322178,Why we can see virtual image?,<p>Why can we see virtual images? It cannot be...,<optics><geometric-optics>
296,4068,Formalizing Quantum Field Theory,<p>I'm wondering about current efforts to prov...,<quantum-field-theory><mathematical-physics><r...
297,219909,Formation of Saturn's Rings,<p>Assuming the rings of Saturn started out as...,<solar-system>
298,281740,Could dark energy just be another characterist...,<p>Recent observations of the accelerating exp...,<gravity><cosmology><symmetry><dark-energy>


In [14]:
print("DataSet")
Q2

DataSet


,OId,OTitle,OBody,OTags
0,1,What is spin as it relates to subatomic partic...,<p>I often hear about subatomic particles havi...,<quantum-mechanics><particle-physics><angular-...
1,7,Will Determinism be ever possible?,<p>What are the main problems that we need to ...,<quantum-mechanics><quantum-interpretations><h...
2,9,Why should an action integral be stationary? O...,<p>Hamilton's principle states that a dynamic ...,<lagrangian-formalism><variational-principle><...
3,15,What experiment would disprove string theory?,<p>I know that there's big controversy between...,<string-theory><experimental-physics>
4,17,Why does the sky change color? Why is the sky ...,<p>Why does the sky change color? Why is the s...,<optics><visible-light><scattering><atmospheri...
...,...,...,...,...
6105,678359,How is the answer to this question $1/c$ seconds?,"<p>This is <a href=""https://www.youtube.com/wa...",<electromagnetism><speed-of-light><conductors>...
6106,679619,How do scientists derive equations and formula...,<p>How do scientists use arithmetic to derive ...,<mathematics>
6107,680321,Forces and magnetic lines of force,<p>Magnet releases the magnetic lines of force...,<forces><newtonian-gravity><magnetic-fields><e...
6108,680400,Escape velocity at an angle near a black hole,<p>What's the formula for the velocity require...,<black-holes><escape-velocity>


In [15]:

# # process the body wit tok, stem and modify tags and export

for i in range(Q2.shape[0]):
    
    x = TokStem(hbt(Q2.loc[i,'OBody']),1)
    y = rmTags(Q2.loc[i,'OTags'])
    z = TokStem(hbt(Q2.loc[i,'OTitle']),1)

    Q2.loc[i,'OBody'] = x
    Q2.loc[i,'OTags'] = y
    Q2.loc[i,'OTitle'] =z
  
for i in range(Q1.shape[0]):
    x = TokStem(hbt(Q1.loc[i,'DBody']),1)
    y = rmTags(Q1.loc[i,'DTags'])
    z = TokStem(hbt(Q1.loc[i,'DTitle']),1)

    Q1.loc[i,'DBody'] = x
    Q1.loc[i,'DTags'] = y
    Q1.loc[i,'DTitle'] =z

# Q2.to_csv('body_stem.csv') # export to CSV

# LDA processing

In [16]:
# also create master data for finding topics
data_words = []

for i in range(Q2.shape[0]):
  data_words.append((Q2.loc[i,'OBody']+Q2.loc[i,'OTitle']).split())

# create dictionary
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
# LDA imp    
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=100,minimum_probability=0.0)
lda_train = np.zeros((len(data_words),100)) # Q1 + Q2 size by 100 topics, refer to perplexity in paper

for i in range(len(data_words)): 
	lda_train[i] = np.array(lda_model[corpus[i]])[:,1]



/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


# Cosine Similarity between two strings

In [22]:

def cos_sim(a,b): # finding cosine similarity score for given 2 strings
    # input two strings that are split into words
    us = [] # creating union set
    us += b
    for x in a:
        if x not in b:
            us.append(x)
    freqa,freqb,frequ = dict(),dict(),dict()
    wa,wb = [], []
    for word in a: # freq bag for string a
        if word in freqa:
            freqa[word] += 1
        else:
            freqa[word] = 1
    for word in b:# freq bag for string b
        if word in freqb:
            freqb[word] += 1
        else:
            freqb[word] = 1
    for word in us: # freq bag for union of a and b
        if word in frequ:
            frequ[word] += 1
        else:
            frequ[word] = 1
    for i in range(len(us)): # calc TitleVec for a and b
        x = us[i]
        if x in a:
            wa.append(freqa[x]/frequ[x])
        else:
            wa.append(0)
        if x in b:
            wb.append(freqb[x]/frequ[x])
        else:
            wb.append(0)
    wa = np.array(wa)
    wb = np.array(wb)
    ans = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))
    if np.isnan(ans):
      return 0
    return  ans# cosine value
print(cos_sim(Q2.loc[0,'OTags'].split(),Q1.loc[0,'DTags'].split()))

1.0


## Calculating Similarity component scores

In [24]:

b= ['OTitle','OBody','OTags']
a = ['DTitle','DBody','DTags']
SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
y = 162130
n2 = Q1.shape[0]
wa = np.zeros(100)
wb = np.zeros(100)
for i in range(Q2.shape[0]):
    print(i)
    for j in range(Q1.shape[0]):

        SC[i][j][0] = cos_sim(Q1.loc[j,a[0]].split(),Q2.loc[i,b[0]].split()) # TitleSC
        SC[i][j][1] = cos_sim(Q1.loc[j,a[1]].split(),Q2.loc[i,b[1]].split()) # BodySC
        SC[i][j][2] = cos_sim(Q1.loc[j,a[2]].split(),Q2.loc[i,b[2]].split()) # TagsSC
        text = (Q1.loc[j,'DBody']+Q1.loc[j,'DTitle']).split()
        corp = id2word.doc2bow(text)
        wa, wb = np.array(lda_model[corp])[:,1], lda_train[i] # lda train has Q1 qns and then Q2 qns
        SC[i][j][3] = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        if (np.isnan(SC[i][j][3])):
          SC[i][j][3] = 0
        if (j==35):
          print(SC[i][j])
np.save("Training set Similarity scoresPhysics.npy",SC)


Streaming output truncated to the last 5000 lines.
3610
[0.         0.         0.         0.00095353]
3611
[0.20412415 0.05838742 0.         0.00083017]
3612
[0.         0.06465082 0.         0.00126514]
3613
[0.         0.         0.31622777 0.00151964]
3614
[0.         0.05315774 0.31622777 0.00089842]
3615
[0.         0.03276781 0.         0.00117638]
3616
[0.         0.04828045 0.31622777 0.0613473 ]
3617
[0.         0.04351941 0.         0.00178542]
3618
[0.         0.02654659 0.31622777 0.0014064 ]
3619
[0.         0.05309319 0.         0.001153  ]
3620
[0.        0.        0.        0.0015441]
3621
[0.         0.03868392 0.         0.00056017]
3622
[0.         0.05647825 0.         0.00098167]
3623
[0.         0.07284644 0.5        0.00107148]
3624
[0.         0.01932841 0.         0.00101414]
3625
[0.         0.11258799 0.         0.49855848]
3626
[0.         0.04222003 0.         0.04262704]
3627
[0.         0.03649659 0.         0.00066709]
3628
[0.         0.05970814 0.35355

# Estimating parameters

In [26]:
SC = np.load("Training set Similarity scoresPhysics.npy")

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count

ga,gb,gc,gd = 0,0,0,0 # final parameter values
for a in sp:
    for b in sp:
        for c in sp:
            for d in sp:

                if a==0 and b==0 and c==0 and d==0:
                    continue
                tc = 0 # temp count
                for i in range(Q1.shape[0]): # iterate all duplicate qns
                    te = []
                    for j in range(Q2.shape[0]): # iterate over available qns
                        te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]+d*SC[j][i][3]) # composer score
                    x = te.index(max(te)) # get index of maximum element
                    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
                        tc += 1
                if tc >= gco: # optimal value found
                    ga,gb,gc,gd = a,b,c,d
                    gco = tc
                    print(ga,gb,gc,gd,tc)


In [28]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
ga,gb,gc,gd = 0,0,0,0 # final parameter values
import random
k =  20

random_composers = np.zeros((100,5)) # store composer values, and recall rate
for iter in range(0,100):
  a = random.uniform(0,1)
  b = random.uniform(0,1)
  c = random.uniform(0,1)
  d = random.uniform(0,1)
  if a==0 and b==0 and c==0 and d ==0:
      continue
  tc = 0 # temp count
  count = 0
  de = {}
  for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
      te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2] +d*SC[j][i][3]) # composer score
      de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #   tc += 1

    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    
  random_composers[iter] = [a,b,c,d,count]
  print(count)
  if count >= gco: # optimal value found
    ga,gb,gc,gd = a,b,c,d
    gco = count
    print(ga,gb,gc,gd,count)
#             print(tc)

np.save("composer_score_recallPhysics.npy",random_composers)

174
0.5457064824674146 0.5716209041710257 0.5014430850680104 0.7863915122451192 174
218
0.9765315351584902 0.558093138220439 0.5704549747971008 0.4076652128038316 218
228
0.3805223601758816 0.8444796728864523 0.6470614649794051 0.06036115578434531 228
111
106
225
199
125
146
197
83
198
212
165
207
211
151
200
205
106
213
147
93
227
163
183
155
180
208
191
98
191
219
223
218
205
196
203
189
113
223
174
156
195
124
221
214
152
219
202
200
149
174
146
154
159
125
205
169
185
206
223
200
218
219
230
0.6551833822319766 0.6131337000089024 0.43429301732332015 0.2134832135233694 230
166
218
226
208
159
212
188
226
223
134
111
203
153
171
222
140
176
216
171
234
0.48044852015417805 0.9271799566568408 0.5299574682215845 0.009443980128142515 234
192
190
219
201
216
206
211
114
141
224
159
199
114
215


In [30]:
arr = np.load("composer_score_recall.npy")
comp_scores = pd.Datatframe(arr)
# np.savetxt("composer_score_recall_float.npy",arr, fmt='%f')

[[5.45706482e-01 5.71620904e-01 5.01443085e-01 7.86391512e-01
  1.74000000e+02]
 [9.76531535e-01 5.58093138e-01 5.70454975e-01 4.07665213e-01
  2.18000000e+02]
 [3.80522360e-01 8.44479673e-01 6.47061465e-01 6.03611558e-02
  2.28000000e+02]
 [5.39608845e-01 8.17379533e-01 1.38705175e-01 9.40762031e-01
  1.11000000e+02]
 [4.53900390e-01 3.18578504e-01 2.79731314e-01 9.24510287e-01
  1.06000000e+02]
 [6.39426491e-01 7.50876011e-01 7.99414989e-01 3.23618897e-01
  2.25000000e+02]
 [2.90498496e-01 1.13143866e-01 5.51768912e-01 3.82495805e-01
  1.99000000e+02]
 [9.36565027e-02 9.23038295e-01 3.35785493e-02 3.60549273e-01
  1.25000000e+02]
 [1.82301977e-02 4.05217282e-01 4.96152653e-01 6.28186423e-01
  1.46000000e+02]
 [6.15906162e-01 9.25826856e-02 3.09004053e-01 3.76529989e-01
  1.97000000e+02]
 [9.95406304e-02 8.18121686e-01 9.96856199e-02 7.36629202e-01
  8.30000000e+01]
 [4.90253310e-01 7.43237170e-01 7.42499526e-01 7.49796453e-01
  1.98000000e+02]
 [2.83277316e-01 8.39806681e-01 8.510864

In [37]:
print("Composer score weights - Alpha: ",ga,"Beta: ",gb,"Tags: ",gc,"Topic: ",gd)

Composer score weights - Alpha:  0.48044852015417805 Beta:  0.9271799566568408 Tags:  0.5299574682215845 Topic:  0.009443980128142515


## Validating the composer model

In [ ]:
tc = 0 # temp count
k =  20
count = 0

print(ga,gb,gc,gd)
original_data = df[['OId','OTitle','OBody','OTags']].loc[300:500-1].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[300:500-1].copy() # doop qns
for i in range(300,300+test_data.shape[0]): # iterate all duplicate qns
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(test_data.loc[i,'DBody']),0)
    y = rmTags(test_data.loc[i,'DTags']).split()
    z = TokStem(hbt(test_data.loc[i,'DTitle']),0)
    
    # text = (x+ y)

    # corp = id2word.doc2bow(text)

    # wa, wb = np.array(lda_model[corp])[:,1], lda_train[i]
    de = {}    
    b= ['OTitle','OBody','OTags']
    for j in range(Q2.shape[0]): # iterate over available qns
        # wa, wb = np.array(lda_model[corp])[:,1], lda_train[j] # lda train has Q1 qns and then Q2 qns
        # topic_similarity = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        # if (np.isnan(topic_similarity)):
        #   topic_similarity = 0
        te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split()) ) # composer score
        de[j] = te[j]

    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    print(count)


In [34]:
print("Accuracy: ", count/200)

Accuracy:  0.77


## Effect of individual components on recall rates

In [ ]:

tc = 0 # temp count
k =  20
count = [0,0,0] # 5,10,20
ga,gb,gc = 1, 1, 1 # normalised
# print(ga,gb,gc,gd)
original_data = df[['OId','OTitle','OBody','OTags']].loc[300:200-1].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[300:200-1].copy() # doop qns
for i in range(300,300+test_data.shape[0]): # iterate all duplicate qns
    de = {}
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(test_data.loc[i,'DBody']),0)
    y = rmTags(test_data.loc[i,'DTags']).split()
    z = TokStem(hbt(test_data.loc[i,'DTitle']),0)
    
    # text = (x+ y)

    # corp = id2word.doc2bow(text)

    # wa, wb = np.array(lda_model[corp])[:,1], lda_train[i]
        
    b= ['OTitle','OBody','OTags']
    for j in range(Q2.shape[0]): # iterate over available qns
    #     wa, wb = np.array(lda_model[corp])[:,1], lda_train[j] # lda train has Q1 qns and then Q2 qns
    #     topic_similarity = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
    #     if (np.isnan(topic_similarity)):
    #       topic_similarity = 0
        # te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split()) + gd*topic_similarity) # composer score
        te.append(cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
        de[j] = te[j]

    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1
    temp_count= 0
    iter_temp = sorted(de.items(), key= lambda item: item[1], reverse=True)
    ### For k = 5
    k = 5
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[0]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@5 - ",count[0])
    ###m For k = 10
    temp_count = 0
    k = 10
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[1]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@10 : ",count[1])
     ###m For k = 20
    k = 20
    temp_count = 0
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[2]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@20 :- ",count[2])
print("Recall-Rate@5= ", count[0]/200)
print("Recall-Rate@10= ", count[1]/200)
print("Recall-Rate@20= ", count[2]/200)

## Return top k questions

In [40]:
tc = 0 # temp count
k =  20
count = 0
de = {}
print(ga,gb,gc)
no = np.random.randint(300,500)
original_data = df[['OId','OTitle','OBody','OTags']].loc[no].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[no].copy() # doop qns

te = []
# print(test_data.loc[i,'DTags'])
x = TokStem(hbt(df.loc[no,'DBody']),0)
y = rmTags(df.loc[no,'DTags']).split()
z = TokStem(hbt(df.loc[no,'DTitle']),0)
# print(x)
# print(y)
# print(z)
# break
b= ['OTitle','OBody','OTags']
# print(b)
# print(ga,gb,gc)
for j in range(Q2.shape[0]): # iterate over available qns
    te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
    de[j] = te[j]
# x = te.index(max(te)) # get index of maximum element
# if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
#     tc += 1
temp_count= 0
print("------------ Given question ------")
print(no)
print("Title -->  ",df.loc[no,'DTitle'])
print("Question --> ", df.loc[no,'DBody'])
print("Tags --> ", df.loc[no,'DTags'])
print("-----")
for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
  print(" ----- Question ---- ", temp_count)
  print("Title --->", df.loc[key,'OTitle'])
  print("Question --->", df.loc[key,'OBody'])
  print("Tags --->", df.loc[key,'OTags'])
  print(Q2.loc[key], df.loc[no])
  print("----")
  if (temp_count<k):
    if Q2.loc[key,'OId'] == df.loc[no,'OId']:
      print(" ----- Question ---- ", temp_count)
      print("Title --->", df.loc[key,'OTitle'])
      print("Question --->", df.loc[key,'OBody'])
      print("Tags --->", df.loc[key,'OTags'])
      print("----")
          
      count+=1
      break
    temp_count+=1
  else:
    break
print(count)
print(tc,count)
if tc >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,tc)

0.48044852015417805 0.9271799566568408 0.5299574682215845
------------ Given question ------
398
Title -->   Deriving $PV^n = \text{constant}$ for polytropic processes?
Question -->  <p>I am trying to understand a derivation of the formula <span class="math-container">$PV^n = \text{constant}$</span> as given to me by my uni teacher.</p>

<p>Knowing that heat capacity <span class="math-container">$C = dQ/dT \implies dQ = CdT$</span></p>

<p>and the first law of thermodynamics stated as <span class="math-container">$dQ = dU + dW = dU + pdV$</span></p>

<p>we get</p>

<p><span class="math-container">$$ CdT = dU + pdV $$</span></p>

<p>I more or less understand until this point. But then, my notes go:</p>

<p><span class="math-container">$$
CdT =
\left(\frac{dU}{dT}\right)_V dT
+ \left[ \left(\frac{dU}{dV}\right)_T + P \right] dV
$$</span></p>

<p><em>(the subscripts indicate constant volume or temperature)</em></p>

<p>I can’t wrap my head around this. Going back from the equation above I